<a href="https://colab.research.google.com/github/AcerPing/AceTibaMe/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

In [ ]:
import os 
import glob
import pandas as pd

def getdata(mid):
  dn = os.path.dirname(dataset)
  posfn = glob.glob(os.path.join(dn,"aclImdb",mid,"pos","*"))
  negfn = glob.glob(os.path.join(dn,"aclImdb",mid,"neg","*"))
  contents = []
  for fn in posfn + negfn:
    with open(fn,encoding="utf-8") as f:
      contents.append(f.read())
  df = pd.DataFrame({
      "content":contents,
      "sentiment":[1]*len(posfn) + [0]*len(negfn)})
  return df




In [ ]:
train_df = getdata("train")
test_df = getdata("test")
train_df
# test_df

In [ ]:
dataset

In [ ]:
#Tokenizer:把詞化成數字
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["content"])

In [ ]:
#0不要用掉，0是拿來padding的
tok.word_index

In [ ]:
#sequences:數字序列
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])
pd.DataFrame(x_train_seq)

In [ ]:
#截長補短到每一篇文章都是同樣詞數
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=512)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)
pd.DataFrame(x_train_pad)

In [17]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Dropout
INPUT_DIM = 3001
OUTPUT_DIM = 128
INPUT_LENGTH = 512
layers = [Embedding(INPUT_DIM,OUTPUT_DIM,mask_zero=True,input_length=INPUT_LENGTH),
      Flatten(),
      Dense(256,activation="relu"),
      Dropout(0.25),
      Dense(2,activation="softmax")]
model = Sequential(layers)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 512, 128)          384128    
_________________________________________________________________
flatten_2 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               16777472  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 514       
Total params: 17,162,114
Trainable params: 17,162,114
Non-trainable params: 0
_________________________________________________________________


In [19]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [20]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("embedding.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)       
]
model.fit(x_train_pad, 
     y_train, 
     batch_size=200,
     validation_split=0.1,
     epochs=50,
     verbose=2,
     callbacks=callbacks)

Epoch 1/50
113/113 - 7s - loss: 0.6698 - accuracy: 0.6885 - val_loss: 0.4025 - val_accuracy: 0.8144
Epoch 2/50
113/113 - 7s - loss: 0.2189 - accuracy: 0.9144 - val_loss: 0.3652 - val_accuracy: 0.8388
Epoch 3/50
113/113 - 6s - loss: 0.0906 - accuracy: 0.9723 - val_loss: 0.5351 - val_accuracy: 0.8036
Epoch 4/50
113/113 - 6s - loss: 0.0222 - accuracy: 0.9967 - val_loss: 0.6132 - val_accuracy: 0.8152
Epoch 5/50
113/113 - 6s - loss: 0.0052 - accuracy: 0.9997 - val_loss: 0.5609 - val_accuracy: 0.8432
Epoch 6/50
113/113 - 6s - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.6595 - val_accuracy: 0.8368
Epoch 7/50
113/113 - 6s - loss: 9.1679e-04 - accuracy: 1.0000 - val_loss: 0.6941 - val_accuracy: 0.8388


In [21]:
model.evaluate(x_test_pad,y_test)

782/782 [==============================] - 5s 7ms/step - loss: 0.2965 - accuracy: 0.8754


[0.29646989703178406, 0.8753600120544434]